Mengambil data dari website OLX dan Momotor menggunakan scraping data

SCRAPING DATA WEBSITE MOMOTOR

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
base_url = 'https://momotor.id/motor-bekas?q=motor+vario&regency='
r = requests.get(base_url)
r

<Response [200]>

In [3]:
soup = BeautifulSoup(r.content, 'html.parser')
content = soup.find_all('div', attrs={'class':'umcy-card-container'})
content

[<div class="umcy-card-container"><div class="component-umcy-card-catalog shadow-elevation-2 umcy-card-catalog" id="button_UmcyProductCardundefined"><div class="outer-img-wrapper"><div class="product-img-wrapper-catalog"><div class="component-image-slider-umcy"><div class="swiper-container"><div class="swiper-button-prev"></div><div class="swiper-button-next"></div><div class="swiper-pagination"></div><div class="swiper-wrapper"><div class="swiper-slide swiper-slide-duplicate" data-swiper-slide-index="4"><div class="umcy-item-image-container"><span style="box-sizing:border-box;display:block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:absolute;top:0;left:0;bottom:0;right:0"><noscript><img alt="gambar" class="umcy-item-image" data-nimg="fill" decoding="async" id="img_UmcyProductCard4" loading="lazy" sizes="100vw" src="/_next/image?url=http%3A%2F%2Fres.cloudinary.com%2Fuploadz%2Fimage%2Fupload%2Fv1731947753%2Fmomotor%2Fprod%2

In [4]:
content[5].find('p', attrs={'id':'label_umcyProductundefined'})

<p class="subtitle-2" id="label_umcyProductundefined" style="text-overflow:ellipsis;width:100%;white-space:nowrap;overflow:hidden;text-align:left">HONDA<!-- --> <!-- -->ALN VARIO 160 ABS</p>

In [ ]:
listings = []
for offset in range(0, 2868, 12):
    url = f'{base_url}&offset={offset}'
    r = requests.get(url)
    
    # Memastikan permintaan berhasil
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        content = soup.find_all('div', attrs={'class':'umcy-card-container'})

        for i in content:
            title = i.find('p', attrs={'id':'label_umcyProductundefined'})
            price = i.find('p', attrs={'id':'label_umcyProductPriceundefined'})
            year = i.find('p', attrs={'id':'label_umcyProductYearundefined'})
            mileage = i.find('p', attrs={'id':'label_umcyProductMileageundefined'})
            location = i.find('p', attrs={'id':'label_umcyProductLocationundefined'})
           

            # Mengambil teks dari elemen yang ditemukan
            content_dict = {
                'vehicle_name': title.text if title else None,
                'price': price.text if price else None,
                'year': year.text if year else None,
                'mileage': mileage.text if mileage else None,
                'location': location.text if location else None,
                
            }
            listings.append(content_dict)
    else:
        print(f'Error fetching page with offset {offset}: {r.status_code}')

momotor_df = pd.DataFrame(listings)
momotor_df



,vehicle_name,price,year,mileage,location,image
0,HONDA VARIO 125 CBS ISS,"Rp23,100,000",2024,"2,500 km",Padang Bulan,None
1,HONDA VARIO 125 CBS ISS,"Rp22,100,000",2022,"15,000 km",Padang Bulan,None
2,HONDA VARIO 150 ESP EXCLSV,"Rp15,750,000",2017,"30,000 km",Kota Jakarta Barat,None
3,HONDA VARIO 150 ESP EXCLSV,"Rp20,500,000",2020,"30,000 km",Kota Jakarta Barat,None
4,HONDA VARIO 125 ESP CBS,"Rp18,000,000",2018,"30,000 km",Kota Denpasar,None
...,...,...,...,...,...,...
2863,HONDA VARIO 125 CBS ISS,"Rp21,300,000",2022,"15,000 km",Kota Tangerang,None
2864,HONDA VARIO 150 ESP SPORTY,"Rp23,100,000",2022,"15,000 km",Kota Bandung,None
2865,HONDA VARIO 125 CBS ISS,"Rp21,100,000",2021,"30,000 km",Kab. Deli Serdang,None
2866,HONDA VARIO 125 ESPCBS ISS,"Rp21,100,000",2021,"30,000 km",Kab. Deli Serdang,None


In [8]:
momotor_df['tax'] = 'hidup'
momotor_df['seller_type'] = 'dealer'

In [9]:
momotor_df['manufacturer'] = 'suzuki'
momotor_df['model'] = momotor_df['vehicle_name']

In [10]:
momotor_df = momotor_df.drop('vehicle_name', axis=1)

In [11]:

momotor_df.to_csv('dataset momotor.csv', index=False)

SCRAPPING DATA WEBSITE OLX

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.olx.co.id/motor-bekas_c200/q-VARIO'
driver = webdriver.Chrome()
driver.get(url)

for i in range(100):
    time.sleep(2)

    # Try to find and click the "Muat lainnya" button
    try:
        load_more_button = WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._38O09 > button'))
        )
        load_more_button.click()
        time.sleep(2)  # Wait for the listings to load
    except Exception as e:
        print("No more 'Muat lainnya' button found or failed to click:", e)
        break  # Stop if no more 'Muat lainnya' button is found

    time.sleep(3)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # Find all new links to listings
    links = soup.find_all('a', attrs={'class':'_2cbZ2'})
    link_list = []

    for link in links:
        link_list.append('https://www.olx.co.id' + link.get('href'))

link_list

In [15]:
len(link_list)

101

In [21]:
listings_list1 = []

for link in link_list:  
    driver.get(link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # Extract details (as per your original code)
    try:
        listing_title = soup.find('h1', attrs={'data-aut-id':'itemTitle'}).text
    except:
        listing_title = 'no title description'

    try:
        price = soup.find('span', attrs={'data-aut-id':'itemPrice'}).text
    except:
        price = 'Rp 0'
    
    try:
        location = soup.find('span', attrs={'class':'_1RkZP'}).text
    except:
        location = 'unknown'

    try:
        manufacturer = soup.find('span', attrs={'data-aut-id':'value_make'}).text
    except:
        manufacturer = 'unknown'

    try:
        model = soup.find('span', attrs={'data-aut-id':'value_m_tipe'}).text
    except:
        model = 'unknown'

    try:
        year = soup.find('span', attrs={'data-aut-id':'value_m_year'}).text
    except:
        year = 0

    try:
        mileage = soup.find('span', attrs={'data-aut-id':'value_mileage'}).text
    except:
        mileage = None

    try:
        desc = soup.find('div', attrs={'data-aut-id':'itemDescriptionContent'}).text.lower()
        tax = None
        if 'pajak' in desc:
             tax = "pajak" + " " + desc.split('pajak')[1].split()[0]
    except:
        tax = 'unknown'

    try:
        seller = soup.find('span', attrs={'data-aut-id':'value_m_seller_type'}).text
    except:
        seller = 'unknown'
    
    listing_dict = {'listing_title': listing_title,
                    'price': price,
                    'location': location,
                    'manufacturer': manufacturer,
                    'model': model,
                    'year': year,
                    'mileage': mileage,
                    'tax': tax,
                    'seller_type': seller}
    
    listings_list1.append(listing_dict)

olx_df = pd.DataFrame(listings_list1)

In [22]:
olx_df.to_csv('dataset olx.csv', index=False)
